# Data Manipulation with Expressions: Creating and Modifying Columns

- Add, transform or create new columns using `df.with_columns([...])`
- Common transformations with expression API: arithmetic, string, datetime
- Conditional transfomations with `pl.when().then().otherwise()`

Goal: Appreciate Polars transformation expressions inside `.with_columns([])`


## Expressions in Context: `select` vs `with_columns`


In [2]:
import polars as pl

df = pl.DataFrame(
    {
        "coffee": ["Espresso", "Latte", "Mocha"],
        "price": [2.5, 3.0, 3.5],
        "sold": [10, 8, 6],
    }
)

df

coffee,price,sold
str,f64,i64
"""Espresso""",2.5,10
"""Latte""",3.0,8
"""Mocha""",3.5,6


In [3]:
# expressions:
# - declare recipe
# - no data involved so far
revenue_expression = (pl.col("price") * pl.col("sold")).alias("revenue")
revenue_expression

<Expr ['[(col("price")) * (col("sold")…'] at 0x72E580672B50>

In [4]:
# contexts execute expressions
# `.select()` creates a new DataFrame with selected columns
df.select(revenue_expression)

revenue
f64
25.0
24.0
21.0


In [5]:
# add new column with `.with_columns(...)` context
expanded_df = df.with_columns(revenue_expression)
expanded_df

coffee,price,sold,revenue
str,f64,i64,f64
"""Espresso""",2.5,10,25.0
"""Latte""",3.0,8,24.0
"""Mocha""",3.5,6,21.0


In [6]:
# no-inplace mutation of original DataFrame
df

coffee,price,sold
str,f64,i64
"""Espresso""",2.5,10
"""Latte""",3.0,8
"""Mocha""",3.5,6


In [7]:
# assign multiple columns at once
# fully parallelized!
expanded_df_2 = df.with_columns(
    [
        revenue_expression,
        pl.lit("Berlin").alias("city"),
    ]
)
expanded_df_2

coffee,price,sold,revenue,city
str,f64,i64,f64,str
"""Espresso""",2.5,10,25.0,"""Berlin"""
"""Latte""",3.0,8,24.0,"""Berlin"""
"""Mocha""",3.5,6,21.0,"""Berlin"""


In [8]:
# mind using `.alias()` to name a new column

# ❌ shadows "price"
df_bad = df.with_columns([pl.col("price") * pl.col("sold")])

# ✅ `.alias()` prevents shadowing
df_good = df.with_columns([(pl.col("price") * pl.col("sold")).alias("revenue")])

df_bad, df_good

(shape: (3, 3)
 ┌──────────┬───────┬──────┐
 │ coffee   ┆ price ┆ sold │
 │ ---      ┆ ---   ┆ ---  │
 │ str      ┆ f64   ┆ i64  │
 ╞══════════╪═══════╪══════╡
 │ Espresso ┆ 25.0  ┆ 10   │
 │ Latte    ┆ 24.0  ┆ 8    │
 │ Mocha    ┆ 21.0  ┆ 6    │
 └──────────┴───────┴──────┘,
 shape: (3, 4)
 ┌──────────┬───────┬──────┬─────────┐
 │ coffee   ┆ price ┆ sold ┆ revenue │
 │ ---      ┆ ---   ┆ ---  ┆ ---     │
 │ str      ┆ f64   ┆ i64  ┆ f64     │
 ╞══════════╪═══════╪══════╪═════════╡
 │ Espresso ┆ 2.5   ┆ 10   ┆ 25.0    │
 │ Latte    ┆ 3.0   ┆ 8    ┆ 24.0    │
 │ Mocha    ┆ 3.5   ┆ 6    ┆ 21.0    │
 └──────────┴───────┴──────┴─────────┘)

## ☕ Load a Realistic Dataset (Coffee Sales)


In [9]:
# load dataset
import polars as pl
from pathlib import Path

if Path("coffee_sales.parquet").exists():
    coffee_sales = pl.read_parquet("coffee_sales.parquet")
else:
    # minimal inline dataset so the lesson is self-contained
    coffee_sales = pl.DataFrame(
        {
            # stock keeping unit
            "sku": [
                "ESP-001",  # Espresso
                "ESP-002",  # Double Espresso
                "LAT-001",  # Latte
                "WAT-001",  # Water Still
                "WAT-002",  # Water Sparkling
                "TEA-001",  # Green Tea
                "SPC-001",  # Matcha Latte
            ],
            "drink": [
                "Espresso",
                "Double Espresso",
                "Latte",
                "Water Still",
                "Water Sparkling",
                "Green Tea",
                "Matcha Latte",
            ],
            "price": [3.0, 4.0, 4.0, 2.0, 2.2, 2.5, 4.8],
            "quantity": [1, 2, 1, 3, 1, 3, 1],
            "timestamp": pl.datetime_range(
                start=pl.datetime(2025, 9, 1),
                end=pl.datetime(2025, 10, 7),
                interval="1d",
                eager=True,
            ),
            "city": [
                "Berlin",
                "Austin",
                "New York",
                "Los Angeles",
                "Austin",
                "San Francisco",
                "San Francisco",
            ],
        }
    )

print(f"Length of full DataFrame: {len(coffee_sales)}")
coffee_sales.head(8)


Length of full DataFrame: 1000


sku,drink,price,quantity,timestamp,city
str,str,f64,i64,datetime[μs],str
"""WAT-002""","""Water Sparkling""",2.2,1,2025-09-17 08:15:00,"""New York"""
"""LAT-001""","""Latte""",4.0,3,2025-08-07 21:47:00,"""Los Angeles"""
"""SPC-002""","""Iced Americano""",3.5,3,2025-08-28 01:01:00,"""New York"""
"""ESP-002""","""Double Espresso""",4.0,1,2025-09-02 19:01:00,"""Los Angeles"""
"""TEA-003""","""Chai Latte""",4.0,3,2025-09-11 22:34:00,"""Los Angeles"""
"""TEA-001""","""Green Tea""",2.5,2,2025-09-07 08:51:00,"""Los Angeles"""
"""SPC-001""","""Matcha Latte""",4.8,1,2025-09-14 13:21:00,"""New York"""
"""LAT-002""","""Cappuccino""",3.5,1,2025-09-18 10:06:00,"""Los Angeles"""


## Conditional Logic with `when` / `then` / `otherwise`


In [10]:
# City-based discount rules
# Austin 10%, San Francisco 8%, Berlin 5%, others 0%
discount_expr = (
    pl.when(pl.col("city") == "Austin")
    .then(0.10)
    .when(pl.col("city") == "San Francisco")
    .then(0.08)
    .when(pl.col("city") == "Berlin")
    .then(0.05)
    .otherwise(0.00)
    .alias("discount_pct")
)

coffee_sales_discounts = coffee_sales.with_columns([discount_expr])

coffee_sales_discounts.tail(8)


sku,drink,price,quantity,timestamp,city,discount_pct
str,str,f64,i64,datetime[μs],str,f64
"""WAT-001""","""Water Still""",2.0,3,2025-08-17 12:58:00,"""New York""",0.0
"""LAT-001""","""Latte""",4.0,2,2025-09-22 11:31:00,"""Austin""",0.1
"""LEM-002""","""Iced Latte""",4.5,3,2025-08-01 16:56:00,"""Austin""",0.1
"""ESP-003""","""Americano""",3.3,1,2025-08-15 02:32:00,"""Austin""",0.1
"""WAT-002""","""Water Sparkling""",2.2,1,2025-08-10 05:10:00,"""San Francisco""",0.08
"""LAT-003""","""Flat White""",4.2,3,2025-08-15 13:51:00,"""New York""",0.0
"""LAT-003""","""Flat White""",4.2,2,2025-09-13 15:30:00,"""New York""",0.0
"""LAT-001""","""Latte""",4.0,3,2025-09-23 13:01:00,"""Los Angeles""",0.0


## Arithmetic Columns: Revenue, Net Price, Chaining

In [11]:
discounted = coffee_sales_discounts.with_columns(
    [
        # comparison expression creates boolean mask
        (pl.col("discount_pct") > 0).alias("discount_active"),
        # Arithmetic demos: *, -, /
        (pl.col("price") * pl.col("quantity")).alias("gross_revenue"),
        (pl.col("price") * (1 - pl.col("discount_pct"))).alias("net_price"),
    ]
)

discounted.select(
    "discount_active",
    "city",
    "quantity",
    "price",
    "gross_revenue",
    "discount_pct",
    "net_price"
)

discount_active,city,quantity,price,gross_revenue,discount_pct,net_price
bool,str,i64,f64,f64,f64,f64
false,"""New York""",1,2.2,2.2,0.0,2.2
false,"""Los Angeles""",3,4.0,12.0,0.0,4.0
false,"""New York""",3,3.5,10.5,0.0,3.5
false,"""Los Angeles""",1,4.0,4.0,0.0,4.0
false,"""Los Angeles""",3,4.0,12.0,0.0,4.0
…,…,…,…,…,…,…
true,"""Austin""",1,3.3,3.3,0.1,2.97
true,"""San Francisco""",1,2.2,2.2,0.08,2.024
false,"""New York""",3,4.2,12.6,0.0,4.2


In [12]:
# chaining makes intermediate column results available for
# the next context
discounted.with_columns(
    [
        # discount per item
        (pl.col("price") - pl.col("net_price")).alias("discount_amount"),
        # 
        (pl.col("net_price") * pl.col("quantity")).alias("net_revenue"),
    ]
).with_columns(
    [
        # Simple per-unit division demo
        (pl.col("net_revenue") / pl.col("quantity")).alias("avg_net_price_per_unit"),
    ]
).select(
    "quantity",
    "price",
    "discount_active",
    "discount_amount",
    "net_price",
    "gross_revenue",
    "net_revenue",
    "avg_net_price_per_unit",
)


quantity,price,discount_active,discount_amount,net_price,gross_revenue,net_revenue,avg_net_price_per_unit
i64,f64,bool,f64,f64,f64,f64,f64
1,2.2,false,0.0,2.2,2.2,2.2,2.2
3,4.0,false,0.0,4.0,12.0,12.0,4.0
3,3.5,false,0.0,3.5,10.5,10.5,3.5
1,4.0,false,0.0,4.0,4.0,4.0,4.0
3,4.0,false,0.0,4.0,12.0,12.0,4.0
…,…,…,…,…,…,…,…
1,3.3,true,0.33,2.97,3.3,2.97,2.97
1,2.2,true,0.176,2.024,2.2,2.024,2.024
3,4.2,false,0.0,4.2,12.6,12.6,4.2


In [13]:
# Finally remember you can also run all manipulations in parallel
single_chain = (
    coffee_sales.with_columns(
        [
            discount_expr,
        ]
    )
    .with_columns(
        [
            (pl.col("discount_pct") > 0).alias("discount_active"),
            # Arithmetic demos: *, -, /
            (pl.col("price") * pl.col("quantity")).alias("gross_revenue"),
            (pl.col("price") * (1 - pl.col("discount_pct"))).alias("net_price"),
        ]
    )
    .with_columns(
        [
            (pl.col("price") - pl.col("net_price")).alias("discount_amount"),
            (pl.col("net_price") * pl.col("quantity")).alias("net_revenue"),
        ]
    )
    .with_columns(
        [
            # Simple per-unit division demo
            (pl.col("net_revenue") / pl.col("quantity")).alias(
                "avg_net_price_per_unit"
            ),
        ]
    )
)

single_chain.select(
    "quantity",
    "price",
    "discount_active",
    "discount_amount",
    "net_price",
    "gross_revenue",
    "net_revenue",
    "avg_net_price_per_unit",
)

quantity,price,discount_active,discount_amount,net_price,gross_revenue,net_revenue,avg_net_price_per_unit
i64,f64,bool,f64,f64,f64,f64,f64
1,2.2,false,0.0,2.2,2.2,2.2,2.2
3,4.0,false,0.0,4.0,12.0,12.0,4.0
3,3.5,false,0.0,3.5,10.5,10.5,3.5
1,4.0,false,0.0,4.0,4.0,4.0,4.0
3,4.0,false,0.0,4.0,12.0,12.0,4.0
…,…,…,…,…,…,…,…
1,3.3,true,0.33,2.97,3.3,2.97,2.97
1,2.2,true,0.176,2.024,2.2,2.024,2.024
3,4.2,false,0.0,4.2,12.6,12.6,4.2


## String Manipulation with `.str` (patterns, regex, case)

Expression API availalable under `expr.str.` attribute.

Full overview: http://docs.pola.rs/api/python/stable/reference/expressions/string.html


In [14]:
# check string pattern
selected = coffee_sales.select(
    pl.col("drink"),
    pl.col("sku"),
    pl.col("sku").str.starts_with("WAT").alias("is_water"),
    pl.col("sku").str.ends_with("002").alias("second_of_class"),
    pl.col("sku").str.contains("001").alias("first_item_of_class"),
    # contains with . as placeholder
    pl.col("sku").str.contains("AT...1").alias("first_water_or_latte"),
)
selected.tail(10)

drink,sku,is_water,second_of_class,first_item_of_class,first_water_or_latte
str,str,bool,bool,bool,bool
"""Chai Latte""","""TEA-003""",false,false,false,false
"""Water Still""","""WAT-001""",true,false,true,true
"""Water Still""","""WAT-001""",true,false,true,true
"""Latte""","""LAT-001""",false,false,true,true
"""Iced Latte""","""LEM-002""",false,true,false,false
"""Americano""","""ESP-003""",false,false,false,false
"""Water Sparkling""","""WAT-002""",true,true,false,false
"""Flat White""","""LAT-003""",false,false,false,false
"""Flat White""","""LAT-003""",false,false,false,false


In [15]:
# regex for extraction, pattern matching or replacement
result = coffee_sales.select(
    pl.col("sku").str.extract(r"(\d+)").alias("class_item"),
    pl.col("sku").str.replace("SPC", "SPECIAL").alias("special"),
)
result

class_item,special
str,str
"""002""","""WAT-002"""
"""001""","""LAT-001"""
"""002""","""SPECIAL-002"""
"""002""","""ESP-002"""
"""003""","""TEA-003"""
…,…
"""003""","""ESP-003"""
"""002""","""WAT-002"""
"""003""","""LAT-003"""


In [16]:
# string manipulations
lowercase_drinks = pl.col("drink").str.to_lowercase().alias("drink_lowercase")
lowercase_drinks

<Expr ['col("drink").str.lowercase().a…'] at 0x72E580673450>

In [17]:
coffee_sales.with_columns(
    [
        lowercase_drinks,
        pl.col("drink").str.to_uppercase().alias("drink_uppercase"),
    ]
).select(pl.col("^drink.*$"))

drink,drink_lowercase,drink_uppercase
str,str,str
"""Water Sparkling""","""water sparkling""","""WATER SPARKLING"""
"""Latte""","""latte""","""LATTE"""
"""Iced Americano""","""iced americano""","""ICED AMERICANO"""
"""Double Espresso""","""double espresso""","""DOUBLE ESPRESSO"""
"""Chai Latte""","""chai latte""","""CHAI LATTE"""
…,…,…
"""Americano""","""americano""","""AMERICANO"""
"""Water Sparkling""","""water sparkling""","""WATER SPARKLING"""
"""Flat White""","""flat white""","""FLAT WHITE"""


In [18]:
# string cropping
coffee_sales.select(
    pl.col("sku").str.head(3).alias("sku_classes_v1"),
    pl.col("sku").str.split("-").alias("splitted"),
    pl.col("sku").str.split("-").list.get(0).alias("sku_classes_v2"),
)

sku_classes_v1,splitted,sku_classes_v2
str,list[str],str
"""WAT""","[""WAT"", ""002""]","""WAT"""
"""LAT""","[""LAT"", ""001""]","""LAT"""
"""SPC""","[""SPC"", ""002""]","""SPC"""
"""ESP""","[""ESP"", ""002""]","""ESP"""
"""TEA""","[""TEA"", ""003""]","""TEA"""
…,…,…
"""ESP""","[""ESP"", ""003""]","""ESP"""
"""WAT""","[""WAT"", ""002""]","""WAT"""
"""LAT""","[""LAT"", ""003""]","""LAT"""


In [19]:
# categorize drinks in higher level categories
assign_category = (
    pl.when(pl.col("sku").str.starts_with("ESP") | pl.col("sku").str.starts_with("LAT"))
    .then(pl.lit("Coffee"))
    .when(pl.col("sku").str.starts_with("WAT") | pl.col("sku").str.starts_with("LEM"))
    .then(pl.lit("Bottles"))
    .when(pl.col("sku").str.starts_with("SPC"))
    .then(pl.lit("Special"))
    .otherwise(pl.lit("Other"))
    .alias("category")
)
coffee_sales.with_columns(assign_category).select("sku", "drink", "category")

sku,drink,category
str,str,str
"""WAT-002""","""Water Sparkling""","""Bottles"""
"""LAT-001""","""Latte""","""Coffee"""
"""SPC-002""","""Iced Americano""","""Special"""
"""ESP-002""","""Double Espresso""","""Coffee"""
"""TEA-003""","""Chai Latte""","""Other"""
…,…,…
"""ESP-003""","""Americano""","""Coffee"""
"""WAT-002""","""Water Sparkling""","""Bottles"""
"""LAT-003""","""Flat White""","""Coffee"""


## Temporal Features with `.dt` (weekday, business day)

Expression API availalable under `expr.dt` attribute.

Full overview: https://docs.pola.rs/api/python/stable/reference/expressions/temporal.html


In [20]:
day_of_week = pl.col("timestamp").dt.strftime("%A").alias("day_of_week_string")

coffee_sales.select(
    pl.col('timestamp'),
    day_of_week,
    pl.col("timestamp").dt.strftime("%Y-%m-%d").alias('date_string')
)

timestamp,day_of_week_string,date_string
datetime[μs],str,str
2025-09-17 08:15:00,"""Wednesday""","""2025-09-17"""
2025-08-07 21:47:00,"""Thursday""","""2025-08-07"""
2025-08-28 01:01:00,"""Thursday""","""2025-08-28"""
2025-09-02 19:01:00,"""Tuesday""","""2025-09-02"""
2025-09-11 22:34:00,"""Thursday""","""2025-09-11"""
…,…,…
2025-08-15 02:32:00,"""Friday""","""2025-08-15"""
2025-08-10 05:10:00,"""Sunday""","""2025-08-10"""
2025-08-15 13:51:00,"""Friday""","""2025-08-15"""


In [21]:
coffee_sales.select(
    pl.col("timestamp"),
    day_of_week,
    pl.col("timestamp").dt.weekday().alias("day_of_week"),
    pl.col("timestamp").dt.day().alias("day_of_month"),
    pl.col("timestamp").dt.ordinal_day().alias("day_of_year"),
).sort("timestamp")


timestamp,day_of_week_string,day_of_week,day_of_month,day_of_year
datetime[μs],str,i8,i8,i16
2025-08-01 01:53:00,"""Friday""",5,1,213
2025-08-01 04:42:00,"""Friday""",5,1,213
2025-08-01 08:13:00,"""Friday""",5,1,213
2025-08-01 09:36:00,"""Friday""",5,1,213
2025-08-01 11:19:00,"""Friday""",5,1,213
…,…,…,…,…
2025-09-30 12:54:00,"""Tuesday""",2,30,273
2025-09-30 14:11:00,"""Tuesday""",2,30,273
2025-09-30 15:20:00,"""Tuesday""",2,30,273


In [22]:
# extracting business days
# default:  Monday - Friday without holidays
# custom: weekday mask and/or integrate holidays

coffee_sales.select(

    # assign boolean mask
    pl.col("timestamp").dt.is_business_day().alias("is_business_day"),
    # negate to get weekend
    ~pl.col("timestamp").dt.is_business_day().alias("is_weekend"),

    # create category
    pl.when(pl.col("timestamp").dt.is_business_day())
    .then(pl.lit("Weekday"))
    .otherwise(pl.lit("Weekend"))
    .alias("day_type"),

).tail(10)

is_business_day,is_weekend,day_type
bool,bool,str
false,true,"""Weekend"""
true,false,"""Weekday"""
false,true,"""Weekend"""
true,false,"""Weekday"""
true,false,"""Weekday"""
true,false,"""Weekday"""
false,true,"""Weekend"""
true,false,"""Weekday"""
false,true,"""Weekend"""


##


## 🎁 Bonus Recap and Programmatic Expressions


In [23]:
coffee_sales.with_columns(
    # add conditional category
    assign_category

).filter(
    # use derived category
    pl.col("category").is_in(["Coffee", "Special"]),

    # namespaces work in filter as well
    pl.col("timestamp").dt.is_business_day(),

).select("category","drink", day_of_week).head(10)


category,drink,day_of_week_string
str,str,str
"""Coffee""","""Latte""","""Thursday"""
"""Special""","""Iced Americano""","""Thursday"""
"""Coffee""","""Double Espresso""","""Tuesday"""
"""Coffee""","""Cappuccino""","""Thursday"""
"""Special""","""Iced Americano""","""Friday"""
"""Coffee""","""Espresso""","""Wednesday"""
"""Coffee""","""Cappuccino""","""Wednesday"""
"""Special""","""Iced Americano""","""Thursday"""
"""Special""","""Matcha Latte""","""Tuesday"""


In [24]:
%%time
# programmatic expression generation

# generate 3 scaled price columns without loops
scales = [1.1, 1.2, 1.3, 1.4, 1.5]
exprs = [(pl.col("price") * s).alias(f"price_x{s}") for s in scales]
parallel = coffee_sales.with_columns(exprs)
parallel.select("drink", pl.col("^price.*$")).head()

CPU times: user 9.59 ms, sys: 6.21 ms, total: 15.8 ms
Wall time: 3.92 ms


drink,price,price_x1.1,price_x1.2,price_x1.3,price_x1.4,price_x1.5
str,f64,f64,f64,f64,f64,f64
"""Water Sparkling""",2.2,2.42,2.64,2.86,3.08,3.3
"""Latte""",4.0,4.4,4.8,5.2,5.6,6.0
"""Iced Americano""",3.5,3.85,4.2,4.55,4.9,5.25
"""Double Espresso""",4.0,4.4,4.8,5.2,5.6,6.0
"""Chai Latte""",4.0,4.4,4.8,5.2,5.6,6.0


In [25]:
%%time
# programmatic expression generation
sequential = coffee_sales
for expr in exprs:
    sequential = sequential.with_columns(expr)
sequential.select("drink", pl.col("^price.*$")).head()

CPU times: user 13.3 ms, sys: 26.7 ms, total: 40 ms
Wall time: 9.03 ms


drink,price,price_x1.1,price_x1.2,price_x1.3,price_x1.4,price_x1.5
str,f64,f64,f64,f64,f64,f64
"""Water Sparkling""",2.2,2.42,2.64,2.86,3.08,3.3
"""Latte""",4.0,4.4,4.8,5.2,5.6,6.0
"""Iced Americano""",3.5,3.85,4.2,4.55,4.9,5.25
"""Double Espresso""",4.0,4.4,4.8,5.2,5.6,6.0
"""Chai Latte""",4.0,4.4,4.8,5.2,5.6,6.0


## Wrap-Up


- Use `df.with_columns([...])` to **add or modify** columns.
- Each argument inside is a **Polars expression**, like `pl.col("price") * 2`.
- Use `.alias("new_name")` to name the result.
- Combine multiple expressions in one call — Polars runs them **in parallel**.
- Use `pl.when(...).then(...).otherwise(...)` for **conditional logic** (instead of Python `if/else`).
- String and datetime columns have their own namespaces: `.str` and `.dt`.
- Polars never modifies data in place — always assign the result to a new variable.


In [26]:
# ⚙️ Polars – Transformations & New Columns

# ADD OR MODIFY COLUMNS
# MULTIPLE EXPRESSIONS (computed in parallel)
coffee_sales.with_columns(
    [
        (pl.col("price") * pl.col("quantity")).alias("revenue"),
        (pl.col("price") * 1.1).alias("price_usd"),  # update or replace
    ]
)

# CONDITIONAL LOGIC (vectorized if-else)
coffee_sales.with_columns(
    pl.when(pl.col("city") == "Austin")
    .then(0.10)
    .when(pl.col("city") == "Berlin")
    .then(0.05)
    .otherwise(0.00)
    .alias("discount_pct")
)

# STRING OPERATIONS with `expr.str.` namespace
coffee_sales.with_columns(
    [
        pl.col("drink").str.to_lowercase().alias("drink_lower"),
        pl.col("sku").str.starts_with("ESP").alias("is_espresso"),
    ]
)

# DATE/TIME OPERATIONS with `expr.dt.` namespace
coffee_sales.with_columns(
    [
        pl.col("timestamp").dt.year().alias("year"),
        pl.col("timestamp").dt.strftime("%b").alias("month_abbr"),
    ]
)

# UPDATE EXISTING COLUMNS (by aliasing same name)
coffee_sales = coffee_sales.with_columns((pl.col("price") * 0.9).alias("price"))

print("Done - Happy Coding!")

Done - Happy Coding!
